In [ ]:
def getTag(row):
	uIdx, mIdx, tag, time = row
	return (tag.lower(), 1)

def count(x, y):
	return (x + y)

def checkFrq(tagCount):
	tag, count = tagCount
	return (count > 20)

class frqFilter(object):
	def __init__(self, tagCount):
		self.freqTag = {tag for (tag, count) in tagCount}
		return
	def checkFrq(self, row):
		uIdx, mIdx, tag, time = row
		return (tag.lower() in self.freqTag)

In [ ]:
data = spark.read.csv('/user/hz333/data/project/tags.csv', header = True)

In [ ]:
#(uIdx, mIdx, tag, time) => (tag, 1)
tags = data.rdd.map(getTag)
#(tag, 1) => (tag, count)
tagCount = tags.reduceByKey(count)

In [ ]:
#(tag, count) => (frqTag, count)
frqTag = tagCount.filter(checkFrq)
frqTag = frqTag.collect()

In [ ]:
print(frqTag)

In [ ]:
frqTagF = frqFilter(frqTag)
#(uIdx, mIdx, tag, time) => (uIdx, mIdx, frqTag, time)
frqTagData = data.rdd.filter(frqTagF.checkFrq)

In [ ]:
frqTagCSV = spark.createDataFrame(frqTagData, samplingRatio = 1)
frqTagCSV.repartition(1).write.option('header', 'false').csv('/user/hz333/data/project/frqTag.csv')